In [30]:
survey_name = "American_Trends_Panel_W34"
# survey_name = "American_Trends_Panel_W41"
# survey_name = "American_Trends_Panel_W82"

survey_grid_search_mapping = {
    # "American_Trends_Panel_W34": "date0904_personas_full_personadb_bn_hcbds_hp_grid_search.json",
    # "American_Trends_Panel_W34": "date0904_personas_full_personadb_bn_hcbdeu_grid_search.json",
    "American_Trends_Panel_W34": "date0904_personas_full_personadb_grid_search.json",
    # "American_Trends_Panel_W34": "date0904_personas_full_personadb_bn_hcbic_grid_search.json",
    
    # "American_Trends_Panel_W41": "date0909_personas_full_personadb_bn_hcbds_grid_search.json",
    # "American_Trends_Panel_W41": "date0904_personas_full_personadb_bn_hcbdeu_grid_search.json",
    "American_Trends_Panel_W41": "date0904_personas_full_personadb_bn_hcbic_grid_search.json",
    
    # "American_Trends_Panel_W82": "date0909_personas_full_personadb_bn_hcbds_grid_search.json" 
    "American_Trends_Panel_W82": "date0904_personas_full_personadb_bn_hcbdeu_grid_search.json" 
    # "American_Trends_Panel_W82": "date0904_personas_full_personadb_bn_hcbic_grid_search.json"
}

In [31]:
import os
import json

os.chdir('/home/ubuntu/code/pump_post_midterm/pump')

with open(f'opinions_qa/similar_users/{survey_name}/{survey_grid_search_mapping[survey_name]}') as f:
    search = json.load(f)

import pandas as pd
import tabulate

df = pd.DataFrame(search)

subdf = df[['setting', 'top_k', 'skew_thres', 'most_is_gold_ratio', 'gold_concentration_ratio']].sort_values(by='most_is_gold_ratio', ascending=False)
print(len(subdf))
print(tabulate.tabulate(subdf, headers='keys', tablefmt='psql'))

setting_mapping = {
    "demo_persona": "querydp",
    "demo_only": "querydonly",
    "persona_only": "queryponly",
}

81
+----+--------------+---------+--------------+----------------------+----------------------------+
|    | setting      |   top_k |   skew_thres |   most_is_gold_ratio |   gold_concentration_ratio |
|----+--------------+---------+--------------+----------------------+----------------------------|
|  2 | demo_persona |      80 |           10 |                0.648 |                   0.532919 |
|  1 | demo_persona |      60 |           10 |                0.647 |                   0.534025 |
|  6 | demo_persona |     250 |           10 |                0.646 |                   0.526903 |
| 26 | demo_persona |     350 |            3 |                0.646 |                   0.524607 |
| 28 | persona_only |      60 |           10 |                0.646 |                   0.53764  |
|  9 | demo_persona |      40 |            5 |                0.645 |                   0.536219 |
|  3 | demo_persona |     100 |           10 |                0.644 |                   0.531054 |
|  7 | 

# Output a mapping

In [32]:
idx = 1
date = '0912'

if 'hcbds' in survey_grid_search_mapping[survey_name]: bn = 'bds'
elif 'hcbdeu' in survey_grid_search_mapping[survey_name]: bn = 'bdeu'
elif 'hcbic' in survey_grid_search_mapping[survey_name]: bn = 'bic'
else: bn = 'no'

row = df.iloc[idx]
if bn == 'no':
    mapping_name = f"date{date}_personas_full_personadb_idx{idx}_{setting_mapping[row['setting']]}_trainAll_top{row['top_k']}_skew{row['skew_thres']}_withname.json"
else:
    mapping_name = f"date{date}_personas_full_personadb_bn_hc{bn}_idx{idx}_{setting_mapping[row['setting']]}_trainAll_top{row['top_k']}_skew{row['skew_thres']}_withname.json"
path = f'opinions_qa/similar_users/{survey_name}/{mapping_name}'
print(bn)
print(path)
with open(path, 'w') as f:
    json.dump(row['similar_user_mapping'], f, indent=4)

no
opinions_qa/similar_users/American_Trends_Panel_W34/date0912_personas_full_personadb_idx1_querydp_trainAll_top60_skew10_withname.json


In [3]:
import numpy as np

np.std([51.5, 51.2, 51.9])

0.28674417556808585

# Check value of majority

In [33]:
from src.utils import get_file_from_s3
from collections import Counter

In [34]:
resp_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/responses.csv"))
with open(f'experiment/data/human_resp/{survey_name}/user_test_q_key_mapping.json', 'r') as f:
    user_test_q_key_mapping = json.load(f)
test_user_list = list(user_test_q_key_mapping.keys())
train_user_list = [str(_) for _ in range(len(resp_df)) if str(_) not in test_user_list]

In [35]:
len(train_user_list), len(test_user_list)

(1689, 848)

In [36]:
train_user_list = [int(_) for _ in train_user_list]

In [37]:
cnt = []
for test_user in test_user_list:
    for test_q in user_test_q_key_mapping[test_user]:
        gold = resp_df[test_q].iloc[int(test_user)]

        population = resp_df[test_q][resp_df['index'].isin(train_user_list)].dropna()
        try:
            majority = Counter(population).most_common(1)[0][0]
        except:
            print(Counter(population))
            break
        if majority == gold:
            cnt.append(1)
        else:
            cnt.append(0)
print('Accuracy:', sum(cnt)/len(cnt))

Accuracy: 0.596


# Check how many could be correct

In [38]:
import os
os.chdir("/home/ubuntu/code/pump_post_midterm/pump")

from collections import Counter 

# path = "opinions_qa/review/American_Trends_Panel_W34/date0905_v17_personadb_surveys_bdemo_American_Trends_Panel_W34_haikupred_prompt3_history_demo_persona_rag_run1"
# path = "opinions_qa/review/American_Trends_Panel_W34/date0905_v17_personadb_surveys_bdemo_mostcommon40_American_Trends_Panel_W34_haikupred_prompt3_history_demo_persona_rag_run1"
path = "opinions_qa/review/American_Trends_Panel_W34/date0905_v17_personadb_surveys_bdemo_mostcommon40_American_Trends_Panel_W34_haikupred_prompt3_history_demo_persona_rag_run1"

# Usage example
start_marker = "# Similar users' answers"
# start_marker = "# Similar users' most common answer"
end_marker = "# Your Prediction of the Participant's Response (answer with given options verbatim, no formatting, no punctuations or linebreaks at the end, don't explain)"


def extract_text_between_markers(file_path, start_marker, end_marker):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    gold = lines[-1][len("Gold: "):]

    # Initialize variables to track whether we are within the markers
    capturing = False
    extracted_lines = []

    for line in lines:
        if start_marker in line:
            capturing = True
            continue  # Skip the start marker line itself
        elif end_marker in line:
            capturing = False
            break  # Stop once we reach the end marker line

        if capturing:
            extracted_lines.append(line.strip())  # Strip any surrounding whitespace

    return gold, "\n".join(extracted_lines)

correct, total = 0, 0
for filename in os.listdir(path):
    file_path = f"{path}/{filename}"
    gold, extracted_text = extract_text_between_markers(file_path, start_marker, end_marker)
    most = Counter(extracted_text.split("\n")).most_common(1)[0][0]
    if most == gold: correct += 1
    total += 1
print(correct/total)

0.646
